In [146]:
import folium
import csv
import json
from shapely.geometry import shape
import pandas as pd
# 1. Load Telangana boundary GeoJSON
with open("telangana_ac.geojson", "r", encoding="utf-8") as f:
    telangana_geojson = json.load(f)

# 2. Compute bounds of Telangana for restricting map view
bounds = folium.GeoJson(telangana_geojson).get_bounds()

# 3. Create base map centered on Telangana and restricted to its bounds with satellite tiles
m = folium.Map(
    location=[17.9784, 79.5941],
    zoom_start=7,
    max_bounds=True,
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr='Esri'
)

m.fit_bounds(bounds)
Marker_coordinates = []
# 4. Add Telangana boundary layer with constituency labels
for feature in telangana_geojson["features"]:
    geom = feature["geometry"]
    props = feature["properties"]
    constituency_name = props.get("assembly_1") or props.get("assembly") or "Unknown"

    folium.GeoJson(
        feature,
        style_function=lambda x: {
            "color": "#0000FF",
            "weight": 2,
            "fillOpacity": 0.05
        }
    ).add_to(m)

    # Compute centroid using shapely
    try:
        polygon = shape(geom)
        centroid = polygon.centroid
        lat, lon = centroid.y, centroid.x
        Marker_coordinates.append({"constituency": constituency_name, "latitude": lat, "longitude": lon})
        folium.map.Marker(
            [lat, lon],
            icon=folium.DivIcon(html=f'<div style="font-size:10pt; color:white; text-shadow:1px 1px black">{constituency_name}</div>')
        ).add_to(m)
    except Exception as e:
        print(f"Error computing centroid for {constituency_name}: {e}")

# 5. Load MLA data from CSV
with open("mlas.csv", newline='', encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        try:
            lat = float(row["latitude"])
            lon = float(row["longitude"])
            name = row["name"]
            constituency = row["constituency"]
            party = row["party"]
            photo_url = row["photo_url"]

            # Popup HTML
            popup_html = f"""
            <div style=\"text-align:center; width:180px;\">
                <img src=\"{photo_url}\" style=\"width:100px; height:100px; border-radius:8px; margin-bottom:10px;\"><br>
                <strong>{name}</strong><br>
                <em>{constituency}</em><br>
                <span>{party}</span>
            </div>
            """
            iframe = folium.IFrame(html=popup_html, width=200, height=200)
            popup = folium.Popup(iframe, max_width=250)

            # Add marker
            folium.Marker(
                location=[lat, lon],
                popup=popup,
                icon=folium.Icon(color="blue", icon="user")
            ).add_to(m)
        except Exception as e:
            print(f"Skipping row due to error: {e}")

# 6. Save to HTML file
m.save("telangana_mla_map.html")
pd.DataFrame(Marker_coordinates).to_csv("telangana_constituency_centroids.csv", index=False)
print("✅ Map saved as telangana_mla_map.html")


✅ Map saved as telangana_mla_map.html


In [133]:
import pandas as pd

url = "https://www.telangana.gov.in/legislature/members-of-legislative-assembly/"
df = pd.read_html(url)[0]
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
df.to_csv("telangana_mlas_official.csv", index=False)


In [134]:
from rapidfuzz import process, fuzz
import pandas as pd

# Assume df1 and df2 both have a 'constituency' column

def fuzzy_merge(df1, df2, key1, key2, threshold=70, limit=1):
    matches = df1[key1].apply(
        lambda x: process.extractOne(x, df2[key2], scorer=fuzz.token_sort_ratio)
    )
    
    df1['match_name'] = matches.apply(lambda x: x[0] if x and x[1] >= threshold else None)
    df1['match_score'] = matches.apply(lambda x: x[1] if x else None)

    return df1.merge(df2, left_on='match_name', right_on=key2, how='left')

# Example:
# result = fuzzy_merge(df_mlas, df_geo, 'constituency', 'assembly_1', threshold=90)
df = pd.read_csv("telangana_mlas_official.csv")
df['constituencyLowerCase'] = df['constituency'].str.lower()

df_coordinates = pd.read_csv("telangana_constituency_centroids.csv")
df_coordinates['constituency'] = df_coordinates['constituency'].str.lower()
df_merged = fuzzy_merge(df, df_coordinates, 'constituencyLowerCase', 'constituency', threshold=60)   

In [142]:
import os
import csv
import requests
from ddgs import DDGS

# Output directory
os.makedirs("mla_photos", exist_ok=True)

# Read MLA names and clean up photo saving
with open("mlas.csv", newline='', encoding="utf-8") as f:
    reader = csv.DictReader(f)
    mlas = [
        {"name": row["name"].strip(), "query": row["name"].strip() + " Telangana MLA photograph"}
        for row in reader
    ]

# Use DDGS to search and download images
with DDGS() as ddgs:
    for mla in mlas:
        name = mla["name"]
        print(f"🔍 Searching image for {name}...")
        try:
            results = list(ddgs.images(mla["query"], max_results=1))
        except Exception as e:
            print(f"❌ Search failed for {name}: {e}")
            continue

        if not results:
            print(f"❌ No results for {name}")
            continue

        url = results[0]["image"]
        if not url:
            print(f"❌ No image URL for {name}")
            continue

        print(f"➡️ Found image: {url}")

        # Download the image
        try:
            resp = requests.get(url, timeout=10)
            ext = url.split('.')[-1].split('?')[0] or "jpg"
            filename = f"{name.replace(' ', '_')}.{ext}"
            filepath = os.path.join("mla_photos", filename)

            with open(filepath, "wb") as out:
                out.write(resp.content)

            print(f"✅ Saved: {filepath}")
        except Exception as e:
            print(f"❌ Download error for {name}: {e}")


🔍 Searching image for Sri Anumula Revanth Reddy...
❌ Search failed for Sri Anumula Revanth Reddy: No results found.
🔍 Searching image for Dr. Palvai Harish Babu...
❌ Search failed for Dr. Palvai Harish Babu: Request timed out: RuntimeError('error sending request for url (https://duckduckgo.com/?q=Dr.+Palvai+Harish+Babu+Telangana+MLA+photograph): operation timed out\n\nCaused by:\n    operation timed out')
🔍 Searching image for Smt. Kova Laxmi...
➡️ Found image: https://media.telanganatoday.com/wp-content/uploads/2023/12/Kova-laxmi.jpg
✅ Saved: mla_photos\Smt._Kova_Laxmi.jpg
🔍 Searching image for Sri Vivek Venkat Swamy...
➡️ Found image: https://d2e1hu1ktur9ur.cloudfront.net/wp-content/uploads/2023/12/52.jpg
✅ Saved: mla_photos\Sri_Vivek_Venkat_Swamy.jpg
🔍 Searching image for Sri Gaddam Vinod...
➡️ Found image: http://gaddamvinod.com/assets/img/media/41.jpg
✅ Saved: mla_photos\Sri_Gaddam_Vinod.jpg
🔍 Searching image for Sri Kokkirala Premsagar Rao...
➡️ Found image: https://prsindia.org/

In [145]:
import os
import pandas as pd

# Path to photos directory
photo_dir = "mla_photos"

# Load MLA CSV
df = pd.read_csv("mlas.csv")

# Clean helper to match names
def clean_name(name):
    return name.strip().lower().replace(" ", "_").replace(".", "")

# Create a mapping from cleaned filename to actual file URL
photo_map = {
    clean_name(f.rsplit(".", 1)[0]): f"https://raw.githubusercontent.com/anilreddy488/MLA/main/photos/{f}"
    for f in os.listdir(photo_dir)
    if f.lower().endswith((".jpg", ".jpeg", ".png"))
}

# Update each MLA's photo_url
df["photo_url"] = df["name"].apply(lambda x: photo_map.get(clean_name(x), ""))

# Save updated CSV
df.to_csv("mlas_updated.csv", index=False)
print("✅ Updated MLA photo URLs saved to mlas_updated.csv")


✅ Updated MLA photo URLs saved to mlas_updated.csv
